# Homework #13: Extract-Transform-Load
Purpose of our ETL is to prepare data sets to answer the question: Is the price of soybeans dependent on global fuel prices or weather in top-producing states of Iowa and Illinois?

>**What is import glob?**
> An asterisk (*) matches zero or more characters in a segment of a name. For example, dir/*. import glob for name in glob. > > > glob('dir/*'): print name. The pattern matches every pathname (file or directory) in the directory dir, without recursing > > > further into subdirectories.

In [19]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt


#### Extract CSVs into DataFrames

In [35]:
import glob
import numpy as np
iowa_df = pd.concat(map(pd.read_csv, glob.glob("./*.csv")))
iowa_df.count()




STATION    641580
NAME       641580
DATE       641580
PRCP       626854
TOBS       237537
dtype: int64

In [36]:
iowa_df.head()

,STATION,NAME,DATE,PRCP,TOBS
0,USC00135650,"MONTEZUMA 1 W, IA US",1/2/2014,0.08,NaN
1,USC00135650,"MONTEZUMA 1 W, IA US",1/14/2014,0.05,NaN
2,USC00135650,"MONTEZUMA 1 W, IA US",1/21/2014,0.06,NaN
3,USC00135650,"MONTEZUMA 1 W, IA US",2/5/2014,0.22,NaN
4,USC00135650,"MONTEZUMA 1 W, IA US",2/13/2014,0.02,NaN


In [37]:
iowa_df.tail()

,STATION,NAME,DATE,PRCP,TOBS
92829,USC00130200,"AMES 8 WSW, IA US",9/22/2019,0.96,63.0
92830,USC00130200,"AMES 8 WSW, IA US",9/27/2019,0.06,66.0
92831,USC00130200,"AMES 8 WSW, IA US",9/28/2019,0.09,68.0
92832,USC00130200,"AMES 8 WSW, IA US",9/29/2019,0.38,68.0
92833,USC00130200,"AMES 8 WSW, IA US",9/30/2019,0.04,87.0


In [38]:
iowa_df = iowa_df[(~np.isnan(iowa_df.TOBS)) & (~np.isnan(iowa_df.PRCP))]

In [39]:
print(len(iowa_df))

235615


In [40]:
iowa_df.head()

,STATION,NAME,DATE,PRCP,TOBS
504,USC00134561,"LAKE PARK, IA US",1/1/2014,0.16,-6.0
505,USC00134561,"LAKE PARK, IA US",1/2/2014,0.00,-12.0
506,USC00134561,"LAKE PARK, IA US",1/3/2014,0.00,7.0
507,USC00134561,"LAKE PARK, IA US",1/4/2014,0.00,8.0
508,USC00134561,"LAKE PARK, IA US",1/5/2014,0.00,-10.0


In [41]:
## Convert "Date" from a string to a date-time-group variable
iowa_df["DATE"] =  pd.to_datetime(iowa_df["DATE"])
iowa_df.dtypes

STATION            object
NAME               object
DATE       datetime64[ns]
PRCP              float64
TOBS              float64
dtype: object

In [42]:
# Rename the column headers
iowa_df = iowa_df.rename(columns={"DATE": "Date",
                                                          "PRCP": "Iowa_Precip",
                                                          "TOBS": "Iowa_Temp"})
iowa_df.head()

,STATION,NAME,Date,Iowa_Precip,Iowa_Temp
504,USC00134561,"LAKE PARK, IA US",2014-01-01,0.16,-6.0
505,USC00134561,"LAKE PARK, IA US",2014-01-02,0.00,-12.0
506,USC00134561,"LAKE PARK, IA US",2014-01-03,0.00,7.0
507,USC00134561,"LAKE PARK, IA US",2014-01-04,0.00,8.0
508,USC00134561,"LAKE PARK, IA US",2014-01-05,0.00,-10.0


In [43]:
iowa_transformed = iowa_df.groupby(['Date']).mean()
iowa_transformed.head()

,Iowa_Precip,Iowa_Temp
Date,,
2014-01-01,0.080442,3.548673
2014-01-02,0.046486,-1.216216
2014-01-03,0.000174,2.843478
2014-01-04,0.003364,23.445455
2014-01-05,0.015321,-0.541284


In [44]:
iowa_transformed.to_csv("../../output/iowa_transformed.csv")


In [ ]:
###Create database connection